# Introduction

This notebook demonstrates the following processes:
1. Downloading loading displacement time series from the EOST and ESMGFZ services.
2. Applying these displacement series as non-tidal loading (NTL) corrections to GNSS position time series.
3. Saving the corrected data in a format compatible with the Hector software for further analysis.

The workflow includes:
- Retrieving loading displacement data.
- Preprocessing and aligning loading data with GNSS station records.
- Applying corrections and generating corrected time series for subsequent analysis.

**Requirements**: Python ≥ 3.8 with packages `pandas`, `numpy`, `matplotlib`  
**Note**: Ensure that file paths to the GNSS and loading data match your directory structure.

**Outputs**:
- Corrected GNSS time series in `.mom` format, ready for Hector processing.
- Each `.mom` file contains a first-line comment indicating the sampling period, followed by space-separated displacement values.

In [1]:
import urllib.request

import pandas as pd
from glob import glob
import os
import json

import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime, timezone, timedelta

# Helper functions

This sectiont includes a set of helper functions to support the processing of station data and the application of NTL corrections.

- **`datetime_to_mjd(date_obj)` / `mjd_to_datetime(mjd)`**  
  Convert between Python `datetime` objects and Modified Julian Dates (MJD). These functions allow consistent time handling when reading, merging, or processing time series data from different sources.

- **`xyz_to_enu(dx, dy, dz, lat_deg, lon_deg)`**  
  Transform ECEF (XYZ) coordinate differences into local East-North-Up (ENU) displacements. Used to convert global Cartesian GNSS positions into local station-centric displacements.

- **`process_station_displacements(station_name, station_data)`**  
  Process GNSS station data by computing relative ECEF displacements from a reference epoch and converting them to ENU coordinates in millimeters. This function prepares the time series for analysis and comparison with non-tidal loading corrections.

**Note**: Not all functions used in this notebook are defined here; some are introduced later in the notebook.

In [2]:
def datetime_to_mjd(date_obj):
    """
    Convert a datetime or ISO-format string to Modified Julian Date (MJD).

    Parameters:
    - date_obj: datetime object or string in "YYYY-MM-DD" or "YYYY-MM-DDTHH:MM:SS" format.

    Returns:
    - mjd: float representing Modified Julian Date
    """
    
    if isinstance(date_obj, str):
        try:
            # Try parsing as ISO format (with time)
            date_obj = datetime.strptime(date_obj, "%Y-%m-%dT%H:%M:%S")
        except ValueError:
            try:
                # Fall back to date-only format
                date_obj = datetime.strptime(date_obj, "%Y-%m-%d")
            except ValueError as e:
                raise ValueError(f"Unsupported date format. Expected '%Y-%m-%dT%H:%M:%S' or '%Y-%m-%d'. Got: {date_obj}") from e
    
    unix_epoch = datetime(1970, 1, 1)
    jd = 2440587.5 + (date_obj - unix_epoch).total_seconds() / 86400
    mjd = jd - 2400000.5
    return mjd
    

def mjd_to_datetime(mjd):
    """
    Convert Modified Julian Date (MJD) to a Python datetime object in UTC.

    Parameters: 
    - mjd: float representing Modified Julian Date

    Returns: 
    - date_obj: datetime object in UTC
    """
    
    mjd = mjd + 2400000.5  # Convert MJD to Julian Date
    unix_epoch = datetime(1970, 1, 1, tzinfo=timezone.utc)
    date_obj = unix_epoch + timedelta(days=(mjd - 2440587.5))  # Convert JD to UTC datetime
    return date_obj  # Return datetime object instead of string
    

def xyz_to_enu(dx, dy, dz, lat_deg, lon_deg):
    """
    Convert ECEF coordinate differences to ENU displacements.

    Parameters:
    - dx, dy, dz: coordinate differences in meters (pandas.Series or arrays)
    - lat_deg, lon_deg: station latitude and longitude in degrees

    Returns: 
    - numpy array of shape (n, 3) with ENU displacements in meters
    """
    
    lat = np.deg2rad(lat_deg)
    lon = np.deg2rad(lon_deg)

    # Rotation matrix for ECEF to ENU conversion
    R = np.array([
        [-np.sin(lon),              np.cos(lon),              0],
        [-np.sin(lat)*np.cos(lon), -np.sin(lat)*np.sin(lon), np.cos(lat)],
        [ np.cos(lat)*np.cos(lon),  np.cos(lat)*np.sin(lon), np.sin(lat)]
    ])

    # Convert Series to numpy arrays and stack them
    dxyz = np.stack([dx.to_numpy(), dy.to_numpy(), dz.to_numpy()], axis=0)
    
    # Calculate ENU coordinates
    enu = R @ dxyz

    return enu.T


def process_station_displacements(station_name, station_data):
    """
    Process station displacement data by converting ECEF coordinates to ENU coordinates and computing
    relative differences from the reference epoch.

    Parameters:
    - station_name: string key for the station (used to look up latitude/longitude)
    - station_data: pandas.DataFrame with columns ['date', 'x', 'y', 'z'] (ECEF coordinates)
    
    Returns:
    - station_data: DataFrame with additional columns ['east', 'north', 'up'] in millimeters.
    """
    # Stations coordinates (lat, lon)
    stations = {
    'ABOA': {'lat': -73.043771, 'lon': -13.407135},
    'SYOG': {'lat': -69.006958, 'lon': 39.583745},
    'VESL': {'lat': -71.673797, 'lon': -2.841783}
    }

    # Extract latitude and longitude from stations dictionary
    lat_deg = stations[station_name]['lat']
    lon_deg = stations[station_name]['lon']

    # Convert date to datetime if necessary and reset index
    station_data['date'] = pd.to_datetime(station_data['date'])
    station_data = station_data.reset_index(drop=True)

    # Calculate reference coordinates (performance implications as you rely on the first entry)
    ref_X, ref_Y, ref_Z = station_data.loc[0, ['x', 'y', 'z']]
    station_data['dX'] = station_data['x'] - ref_X
    station_data['dY'] = station_data['y'] - ref_Y
    station_data['dZ'] = station_data['z'] - ref_Z

    # Calculate ENU displacements and convert to millimeters (assuming inputs are meters)
    enu_displacements = xyz_to_enu(station_data['dX'], station_data['dY'], station_data['dZ'], lat_deg, lon_deg)
    station_data[['east', 'north', 'up']] = enu_displacements * 1000  # Convert to mm

    return station_data


# Downloading NTL data
This section downloads non-tidal loading (NTL) displacement time series from two main services:

1. **[EOST (University of Strasbourg)](http://loading.u-strasbg.fr/surface_gravity.php)**
   - Models: ERA5_IB, ERA5_TUGO, MERRA2, and ECCO2 atmospheric/ocean loading models.  
   - Data is retrieved directly from fixed station URLs for each GNSS site (ABOA, SYOG, VESL). Other stations are also possible, more information from [here](http://loading.u-strasbg.fr/displ_all.php).
   - Output: One `.txt` file per station–model combination, saved in the `EOST/` directory.

2. **[ESMGFZ (GFZ Potsdam)](https://rz-vm480.gfz.de/)**  
   - Models:  
     - **NTAL** – Atmospheric loading (ECMWF operational)
     - **NTOL** – Ocean loading (MPIOM)
     - **HYDL** – Hydrological loading (LSDM)
   - Data is requested using latitude/longitude coordinates for each station and dataset-specific entry IDs.  
   - Time span is divided into three periods (1995–1999, 2000–2009, 2010–present) to match GFZ’s repository structure.  
   - Each period is downloaded as a `.csv` file, then merged into a single combined CSV per station and model type.  
   - Output: Merged CSV files stored in the `ESMGFZ/` directory for later processing.

**Requirements**: Internet connection and write access to the target directories (`EOST/` and `ESMGFZ/`).

In [ ]:
# EOST data addresses
syog_era5ib_link = 'http://loading.u-strasbg.fr/ITRF/CF//ERA5_IB/SYQB_66006S005_NEU.era5'
vesl_era5ib_link = 'http://loading.u-strasbg.fr/ITRF/CF//ERA5_IB/VESL_66009M001_NEU.era5'
aboa_era5ib_link = 'http://loading.u-strasbg.fr/ITRF/CF//ERA5_IB/ABOA_XXXXXXXXX_NEU.era5'

syog_era5tugo_link = 'http://loading.u-strasbg.fr/ITRF/CF//ERA5_TUGO/SYQB_66006S005_NEU.era5'
vesl_era5tugo_link = 'http://loading.u-strasbg.fr/ITRF/CF//ERA5_TUGO/VESL_66009M001_NEU.era5'
aboa_era5tugo_link = 'http://loading.u-strasbg.fr/ITRF/CF//ERA5_TUGO/ABOA_XXXXXXXXX_NEU.era5'

syog_ecco2_link = 'http://loading.u-strasbg.fr/ITRF/CF//ECCO2/SYQB_66006S005_NEU.ecco2'
vesl_ecco2_link = 'http://loading.u-strasbg.fr/ITRF/CF//ECCO2/VESL_66009M001_NEU.ecco2'
aboa_ecco2_link = 'http://loading.u-strasbg.fr/ITRF/CF//ECCO2/ABOA_XXXXXXXXX_NEU.ecco2'

syog_merra2_link = 'http://loading.u-strasbg.fr/ITRF/CF//MERRA2_atm/SYQB_66006S005_NEU_ib.merra2'
vesl_merra2_link = 'http://loading.u-strasbg.fr/ITRF/CF//MERRA2_atm/VESL_66009M001_NEU_ib.merra2'
aboa_merra2_link = 'http://loading.u-strasbg.fr/ITRF/CF//MERRA2_atm/ABOA_XXXXXXXXX_NEU_ib.merra2'

# Request data
urllib.request.urlretrieve(aboa_era5ib_link, 'EOST/ABOA_era5ib.txt')
urllib.request.urlretrieve(aboa_era5tugo_link, 'EOST/ABOA_era5tugo.txt')
urllib.request.urlretrieve(aboa_ecco2_link, 'EOST/ABOA_ecco2.txt')
urllib.request.urlretrieve(aboa_merra2_link, 'EOST/ABOA_merra2.txt')

urllib.request.urlretrieve(syog_era5ib_link, 'EOST/SYOG_era5ib.txt')
urllib.request.urlretrieve(syog_era5tugo_link, 'EOST/SYOG_era5tugo.txt')
urllib.request.urlretrieve(syog_ecco2_link, 'EOST/SYOG_ecco2.txt')
urllib.request.urlretrieve(syog_merra2_link, 'EOST/SYOG_merra2.txt')

urllib.request.urlretrieve(vesl_era5ib_link, 'EOST/VESL_era5ib.txt')
urllib.request.urlretrieve(vesl_era5tugo_link, 'EOST/VESL_era5tugo.txt')
urllib.request.urlretrieve(vesl_ecco2_link, 'EOST/VESL_ecco2.txt')
urllib.request.urlretrieve(vesl_merra2_link, 'EOST/VESL_merra2.txt')


In [ ]:
# Define station coordinates for downloading ESMGFZ data
stations = {
    'ABOA': {'lat': -73.043771, 'lon': -13.407135},
    'SYOG': {'lat': -69.006958, 'lon': 39.583745},
    'VESL': {'lat': -71.673797, 'lon': -2.841783}
}

# Define entry IDs for each dataset and time period
entry_ids = {
    "NTAL": {
        "1995-1999": "ecdac7ab-20c3-4622-8936-847899650473",
        "2000-2009": "045c6abc-dd44-47e5-b8bf-67a1645ced28",
        "2010-now": "394c47ee-32e7-462e-83d0-0142ea060e5b",
    },
    "NTOL": {
        "1995-1999": "a283b887-a162-42d3-a7d0-a797d7dca617",
        "2000-2009": "c7110c00-dfd4-433a-b83d-fd839685b394",
        "2010-now": "91ee389e-30dc-4781-af97-9e5f0908edec",
    },
    "HYDL": {
        "1995-1999":"4050364c-e021-461b-9437-1222a4d32368",
        "2000-2009": "cbdf81e0-cfb4-49de-a141-e17bb103a742",
        "2010-now": "bb87d9a7-ceb3-419d-a296-f739076dd1b2",
    }
}

# Define time periods (10-year intervals)
time_periods = {
    "1995-1999": ("1995-01-01", "1999-12-31"),
    "2000-2009": ("2000-01-01", "2009-12-31"),
    "2010-now": ("2010-01-01", "2024-12-31"),
}

# Define variables
variables = "&variable=duEW&variable=duNS&variable=duV"

# Define base URL format
base_url = "http://rz-vm115.gfz-potsdam.de:8080/repository/entry/show/{}_point"

# Create output directory if it doesn’t exist
output_dir = "ESMGFZ"
os.makedirs(output_dir, exist_ok=True)

# Loop through stations, datasets, and time periods
for station, coords in stations.items():
    lat, lon = coords['lat'], coords['lon']
    
    for dataset, periods in entry_ids.items():
        for period, dates in time_periods.items():
            # Determine the correct entry ID
            entry_id = periods[period]

            # Construct the request URL
            url = (
                f"{base_url.format(period)}"
                f"?submit=Get%20Data&output=data.gridaspoint"
                f"&entryid={entry_id}"
                f"&location.latitude={lat}&location.longitude={lon}"
                f"&calendar=proleptic_gregorian"
                f"&fromdate={dates[0]}%2000%3A00%3A00%20UTC"
                f"&todate={dates[1]}%2021%3A00%3A00%20UTC"
                f"&format=csv{variables}"
            )

            # Define filename
            file_name = f"{output_dir}/{station}/{station}_{dataset}_{period}.csv" 

            # Download the file
            print(f"Downloading {file_name} ...")
            try:
                urllib.request.urlretrieve(url, file_name)
                print(f"Saved: {file_name}")
            except Exception as e:
                print(f"Failed to download {file_name}: {e}")

print("Download complete.")

# Combine the CSV files of each NTL for each station
# Loop through each station folder
for station in os.listdir(output_dir):
    station_path = os.path.join(output_dir, station)

    if os.path.isdir(station_path):  # Ensure it's a directory
        print(f"Processing station: {station}")

        for dataset in ["NTAL", "NTOL", "HYDL"]:
            output_file = os.path.join(station_path, f"{station}_{dataset}_combined.csv")

            # Find all CSV files for the dataset type (e.g., "*_NTAL_*.csv")
            csv_files = glob(os.path.join(station_path, f"*_{dataset}_*.csv"))

            # Handle missing files
            if not csv_files:
                print(f"No {dataset} files found for {station}. Skipping...")
                continue

            # Read and concatenate all CSVs for this dataset type
            dfs = [pd.read_csv(f) for f in csv_files]
            combined_df = pd.concat(dfs, ignore_index=True)

            # Sort by time if needed (assuming time is the first column)
            combined_df.sort_values(by=combined_df.columns[0], inplace=True)

            # Save the merged file
            combined_df.to_csv(output_file, index=False)

print("Merging complete!")


# GNSS data
This section describes the preprocessing of uncorrected GNSS displacement data from multiple datasets. Ensure that the file paths are correct before running the code. The datasets included are:

1. **GR dataset** – Combines four solutions. Data available [here](https://doi.pangaea.de/10.1594/PANGAEA.967516).
2. **NGL dataset** – Available for SYOG and VESL stations. Data available [here](https://geodesy.unr.edu/NGLStationPages/stations/).
3. **TUD and OSU datasets** – Two additional solutions from the GR dataset obtained via personal communication.
5. **AY dataset** – In-house processed data.


For each dataset, the following steps are applied:

- **Data loading** – Read CSV, TXT, or JSON files and assign standard column names.
- **Unit conversion** – Convert displacement values from meters to millimeters where necessary.
- **Time handling** – Convert dates to `datetime` objects, compute Modified Julian Dates (MJD), and filter the data between February 2003 and December 2023.
- **Displacement processing** – For each station, displacements are normalized relative to the first epoch and transformed into local East-North-Up (ENU) components using `process_station_displacements`.

This preprocessing ensures that all GNSS time series are harmonized in units, time reference, and local coordinate representation, ready for subsequent analysis and the application of non-tidal loading corrections.

In [3]:
# GR

column_names = ['date', 'x', 'y', 'z','std_x', 'std_y', 'std_z', 'corr_xy', 'corr_xz', 'corr_yz', 'dis_n', 'dis_e', 'dis_u']

aboa_gr = pd.read_csv('GNSS/ABOA_GR.tab', sep='\t', names=column_names, header=None, skiprows=1)
syog_gr = pd.read_csv('GNSS/SYOG_GR.tab', sep='\t', names=column_names, header=None, skiprows=1)
vesl_gr= pd.read_csv('GNSS/VESL_GR.tab', sep='\t', names=column_names, header=None, skiprows=1)

# Convert displacement to mm
for i, df in enumerate([aboa_gr, syog_gr, vesl_gr]):
    df[['dis_e', 'dis_n', 'dis_u']] *= 1000  # Convert meters to mm
    df['MJD'] = df['date'].apply(lambda x: datetime_to_mjd(x) if pd.notna(x) else None)
    df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert to datetime, handle errors
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df = df[(df['date'] >= '2003-02-01') & (df['date'] <= '2023-12-31')].copy()

    
    # Save back to original variable
    if i == 0:
        aboa_gr = df
    elif i == 1:
        syog_gr = df
    else:
        vesl_gr = df

# Process displacements for ABOA, SYOG, and VESL
aboa_gr = process_station_displacements('ABOA', aboa_gr)
syog_gr = process_station_displacements('SYOG', syog_gr)
vesl_gr = process_station_displacements('VESL', vesl_gr)

syog_gr.head()

,date,x,y,z,std_x,std_y,std_z,corr_xy,corr_xz,corr_yz,dis_n,dis_e,dis_u,MJD,dX,dY,dZ,east,north,up
0,2003-02-01,1.766208e+06,1.460290e+06,-5.932298e+06,0.00234,0.00176,0.00444,0.50032,-0.61248,-0.58647,20.80,-33.60,28.77,52671.5,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000
1,2003-02-02,1.766208e+06,1.460290e+06,-5.932298e+06,0.00396,0.00274,0.00677,0.47935,-0.67041,-0.59432,22.19,-33.29,20.48,52672.5,-0.00148,-0.00082,0.00824,0.311095,1.399275,-8.288887
2,2003-02-03,1.766208e+06,1.460290e+06,-5.932298e+06,0.00250,0.00189,0.00471,0.51028,-0.61594,-0.59748,20.20,-34.01,24.50,52673.5,-0.00135,-0.00164,0.00377,-0.403711,-0.596410,-4.266886
3,2003-02-04,1.766208e+06,1.460290e+06,-5.932298e+06,0.00339,0.00237,0.00598,0.43086,-0.64941,-0.58272,21.21,-34.61,18.69,52674.5,-0.00185,-0.00283,0.00956,-1.002234,0.410172,-10.082276
4,2003-02-05,1.766208e+06,1.460290e+06,-5.932298e+06,0.00238,0.00189,0.00471,0.53215,-0.61119,-0.63715,21.67,-34.68,24.19,52675.5,0.00005,-0.00136,0.00459,-1.080004,0.871288,-4.581992


In [4]:
# NGL
# Column names
columns = ['station', 'date', 'decimal_year', 'x', 'y', 'z', 'std_x', 'std_y', 'std_z', 'corr_xy', 'corr_yz', 'corr_xz', 'antenna_height']

syog_ngl = pd.read_csv('GNSS/SYOG_NGL.txyz2.txt', sep='\\s+', names=columns)
vesl_ngl = pd.read_csv('GNSS/VESL_NGL.txyz2.txt', sep='\\s+', names=columns)

for i, df in enumerate([syog_ngl, vesl_ngl]):
    df['date'] = pd.to_datetime(df['date'], format='%y%b%d', errors='coerce')  # Convert to datetime, handle errors
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df['MJD'] = df['date'].apply(lambda x: datetime_to_mjd(x) if pd.notna(x) else None)
    df = df[(df['date'] >= '2003-02-01') & (df['date'] <= '2023-12-31')].copy()

    # Save back to original variable
    if i == 0:
        syog_ngl = df
    else:
        vesl_ngl = df
        
# Process displacements for SYOG and VESL
syog_ngl = process_station_displacements('SYOG', syog_ngl)
vesl_ngl = process_station_displacements('VESL', vesl_ngl)

syog_ngl.head()


,station,date,decimal_year,x,y,z,std_x,std_y,std_z,corr_xy,corr_yz,corr_xz,antenna_height,MJD,dX,dY,dZ,east,north,up
0,SYOG,2003-02-01,2003.0856,1.766208e+06,1.460290e+06,-5.932298e+06,0.001941,0.001730,0.006017,0.598334,-0.575781,-0.644829,0.0,52671.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,SYOG,2003-02-02,2003.0883,1.766208e+06,1.460290e+06,-5.932298e+06,0.003233,0.002351,0.007927,0.650038,-0.602090,-0.728828,0.0,52672.0,-0.004314,0.005545,0.019248,7.022677,7.090930,-17.895965
2,SYOG,2003-02-03,2003.0910,1.766208e+06,1.460290e+06,-5.932298e+06,0.001977,0.001815,0.006124,0.580927,-0.583285,-0.642008,0.0,52673.0,-0.003306,-0.005278,0.006989,-1.961546,-3.014921,-8.642397
3,SYOG,2003-02-04,2003.0938,1.766208e+06,1.460290e+06,-5.932298e+06,0.002338,0.001969,0.006665,0.623314,-0.608244,-0.650978,0.0,52674.0,-0.008505,-0.006624,0.018677,0.314498,-3.368920,-21.297441
4,SYOG,2003-02-05,2003.0965,1.766208e+06,1.460290e+06,-5.932298e+06,0.001882,0.001646,0.005791,0.603978,-0.604915,-0.641631,0.0,52675.0,-0.000839,-0.003468,0.003262,-2.137691,-1.498268,-4.069134


In [5]:
# TUD
# Column names
column_names = ['station', 'date', 'flag', 'x', 'y', 'z', 'std_X', 'std_Y', 'std_Z','cor_XY', 'cor_XZ', 'cor_YZ', 'weight']

aboa_tud = pd.read_csv('GNSS/ABOA_TUD.xyz', sep='\\s+', names=column_names, skiprows=7)
syog_tud = pd.read_csv('GNSS/SYOG_TUD.xyz', sep='\\s+', names=column_names, skiprows=7)
vesl_tud = pd.read_csv('GNSS/VESL_TUD.xyz', sep='\\s+', names=column_names, skiprows=7)

# Convert date to MJD and filter by date range
for i, df in enumerate([aboa_tud, syog_tud, vesl_tud]):
    df['MJD'] = df['date'].apply(lambda x: datetime_to_mjd(x) if pd.notna(x) else None)
    df = df[(df['date'] >= '2003-02-01') & (df['date'] <= '2023-12-31')].copy()
    
    # Save back to original variable
    if i == 0:
        aboa_tud = df
    elif i == 1:
        syog_tud = df
    else:
        vesl_tud = df

# Process displacements for ABOA, SYOG, and VESL
aboa_tud = process_station_displacements('ABOA', aboa_tud)
syog_tud = process_station_displacements('SYOG', syog_tud)
vesl_tud = process_station_displacements('VESL', vesl_tud)


syog_tud.head()

,station,date,flag,x,y,z,std_X,std_Y,std_Z,cor_XY,cor_XZ,cor_YZ,weight,MJD,dX,dY,dZ,east,north,up
0,66006S002,2003-02-01,W,1.766208e+06,1.460290e+06,-5.932298e+06,0.000249,0.000240,0.000818,0.517034,-0.556598,-0.604015,0.0011,52671.0,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000
1,66006S002,2003-02-02,W,1.766208e+06,1.460290e+06,-5.932298e+06,0.000339,0.000323,0.001140,0.519745,-0.573416,-0.615539,0.0012,52672.0,-0.00025,-0.00034,-0.00070,-0.102735,-0.632933,0.506895
2,66006S002,2003-02-03,W,1.766208e+06,1.460290e+06,-5.932298e+06,0.000267,0.000255,0.000887,0.529202,-0.576091,-0.606027,0.0012,52673.0,0.00059,0.00031,0.00066,-0.137036,0.845398,-0.382522
3,66006S002,2003-02-04,W,1.766208e+06,1.460290e+06,-5.932298e+06,0.000322,0.000257,0.000989,0.507924,-0.631761,-0.541302,0.0012,52674.0,-0.00280,-0.00270,0.01219,-0.296699,0.746158,-12.770329
4,66006S002,2003-02-05,W,1.766208e+06,1.460290e+06,-5.932298e+06,0.000269,0.000296,0.001013,0.548538,-0.580387,-0.686026,0.0011,52675.0,-0.00220,0.00128,-0.00191,2.388340,-1.505766,1.467992


In [6]:
# OSU

# Function to load and process time_series from a JSON file
def load_time_series_json(filepath):
    with open(filepath) as f:
        data = json.load(f)
    ts = data['time_series']
    return  pd.DataFrame({key: ts[key] for key in ['mjd', 'x', 'y', 'z', 'n', 'e', 'u'] if len(ts[key]) == len(ts['mjd'])})

# Load time series data for ABOA, SYOG, and VESL
aboa_osu = load_time_series_json('GNSS/ABOA_OSU.json')
syog_osu = load_time_series_json('GNSS/SYOG_OSU.json')
vesl_osu = load_time_series_json('GNSS/VESL_OSU.json')

# Convert displacement to mm
for i, df in enumerate([aboa_osu, syog_osu, vesl_osu]):
    df[['e', 'n', 'u']] *= 1000  # Convert meters to mm
    df.rename(columns={'mjd': 'MJD'}, inplace=True)
    df['date'] = df['MJD'].apply(lambda x: mjd_to_datetime(x) if pd.notna(x) else None)
    df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert to datetime, handle errors
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df = df[(df['date'] >= '2003-02-01') & (df['date'] <= '2023-12-31')].copy()
    
    # Save back to original variable
    if i == 0:
        aboa_osu = df
    elif i == 1:
        syog_osu = df
    else:
        vesl_osu = df

# Process displacements for ABOA, SYOG, and VESL
aboa_osu = process_station_displacements('ABOA', aboa_osu)
syog_osu = process_station_displacements('SYOG', syog_osu)
vesl_osu = process_station_displacements('VESL', vesl_osu)

vesl_osu.head()

,MJD,x,y,z,n,e,u,date,dX,dY,dZ,east,north,up
0,52671,2.009330e+06,-99741.472907,-6.033158e+06,-53.078679,3.262847,-21.540744,2003-02-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,52672,2.009330e+06,-99741.472502,-6.033158e+06,-53.939590,3.603221,-23.144999,2003-02-02,-0.001303,0.000405,0.001252,0.340374,-0.860911,-1.604255
2,52673,2.009330e+06,-99741.473042,-6.033158e+06,-54.961172,2.973860,-25.745178,2003-02-03,-0.003120,-0.000134,0.003399,-0.288987,-1.882494,-4.204434
3,52674,2.009330e+06,-99741.472225,-6.033158e+06,-55.338377,3.737949,-28.030159,2003-02-04,-0.004157,0.000682,0.005450,0.475102,-2.259698,-6.489415
4,52675,2.009330e+06,-99741.473582,-6.033158e+06,-53.863946,2.454582,-27.659167,2003-02-05,-0.002706,-0.000675,0.005561,-0.808265,-0.785267,-6.118423


In [7]:
# AY
# Column names
column_names = ['decimal-year', 'east', 'north', 'up', 'sigmaE', 'sigmaN', 'sigmaU', 'corrEN', 'corrEU', 'corrNU', 'julian-sec', 'year', 'month', 'day', 'hour', 'minute', 'second']

aboa_ay = pd.read_csv('GNSS/ABOA_AY.series', sep='\\s+', names=column_names)
syog_ay = pd.read_csv('GNSS/SYOG_AY.series', sep='\\s+', names=column_names)
vesl_ay = pd.read_csv('GNSS/VESL_AY.series', sep='\\s+', names=column_names)

# Convert displacement to mm
for i, df in enumerate([aboa_ay, syog_ay, vesl_ay]):
    df[['east', 'north', 'up', 'sigmaE', 'sigmaN', 'sigmaU']] *= 1000   # Convert meters to mm
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']].astype(str).agg('-'.join, axis=1) + ' 12:00:00')
    
    # Group by date first
    df = df.groupby('date', as_index=False).mean(numeric_only=True)
    
    # MJD and date formatting
    df['MJD'] = df['date'].apply(lambda x: datetime_to_mjd(x) if pd.notna(x) else None)
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df = df[(df['date'] >= '2003-02-01') & (df['date'] <= '2023-12-31')].copy()
 
    # Save back to original variable
    if i == 0:
        aboa_ay = df
    elif i == 1:
        syog_ay = df
    else:
        vesl_ay = df


syog_ay.head()

,date,decimal-year,east,north,up,sigmaE,sigmaN,sigmaU,corrEN,corrEU,corrNU,julian-sec,year,month,day,hour,minute,second,MJD
475,2003-02-01,2003.085567,33.302,-27.527,1.286,1.782,2.590,8.914,-0.167028,-0.082621,-0.090367,97373100.0,2003.0,2.0,1.0,12.0,5.0,0.0,52671.5
476,2003-02-05,2003.096500,33.172,-25.718,-1.838,1.857,2.787,9.862,-0.084776,-0.076096,-0.042264,97718100.0,2003.0,2.0,5.0,11.0,55.0,0.0,52675.5
477,2003-02-11,2003.112936,28.484,-27.921,-2.754,1.701,2.593,9.609,0.024209,-0.187838,-0.149175,98236800.0,2003.0,2.0,11.0,12.0,0.0,0.0,52681.5
478,2003-02-12,2003.115665,30.818,-25.187,-10.220,1.778,2.894,9.340,-0.054947,-0.128033,-0.091937,98322900.0,2003.0,2.0,12.0,11.0,55.0,0.0,52682.5
479,2003-02-13,2003.118403,31.007,-24.585,-1.635,1.647,2.367,9.297,0.021599,-0.092969,-0.136406,98409300.0,2003.0,2.0,13.0,11.0,55.0,0.0,52683.5


# NTL data
This section preprocesses NTL data from **EOST** and **ESMGFZ**. The workflow is as follows:
1. Daily NTL displacements are loaded for each station.
2. Dates are converted to datetime, and values to millimeters if necessary.
3. Multiple daily measurements are averaged to obtain a single daily value per component.

In [8]:
# EOST
# Create dataframes
aboa_era5ib = pd.read_csv('EOST/ABOA_era5ib.txt', sep='\\s+', names=['MJD', 'east_ib', 'north_ib', 'up_ib'])
aboa_era5tugo = pd.read_csv('EOST/ABOA_era5tugo.txt', sep='\\s+', names=['MJD', 'east_tugo', 'north_tugo', 'up_tugo'])
aboa_ecco2 = pd.read_csv('EOST/ABOA_ecco2.txt', sep='\\s+', names=['MJD', 'east_ecco2', 'north_ecco2', 'up_ecco2'])
aboa_merra2 = pd.read_csv('EOST/ABOA_merra2.txt', sep='\\s+', names=['MJD', 'east_merra2', 'north_merra2', 'up_merra2'])

syog_era5ib = pd.read_csv('EOST/SYOG_era5ib.txt', sep='\\s+', names=['MJD', 'east_ib', 'north_ib', 'up_ib'])
syog_era5tugo = pd.read_csv('EOST/SYOG_era5tugo.txt', sep='\\s+', names=['MJD', 'east_tugo', 'north_tugo', 'up_tugo'])
syog_ecco2 = pd.read_csv('EOST/SYOG_ecco2.txt', sep='\\s+', names=['MJD', 'east_ecco2', 'north_ecco2', 'up_ecco2'])
syog_merra2 = pd.read_csv('EOST/SYOG_merra2.txt', sep='\\s+', names=['MJD', 'east_merra2', 'north_merra2', 'up_merra2'])

vesl_era5ib = pd.read_csv('EOST/VESL_era5ib.txt', sep='\\s+', names=['MJD', 'east_ib', 'north_ib', 'up_ib'])
vesl_era5tugo = pd.read_csv('EOST/VESL_era5tugo.txt', sep='\\s+', names=['MJD', 'east_tugo', 'north_tugo', 'up_tugo'])
vesl_ecco2 = pd.read_csv('EOST/VESL_ecco2.txt', sep='\\s+', names=['MJD', 'east_ecco2', 'north_ecco2', 'up_ecco2'])
vesl_merra2 = pd.read_csv('EOST/VESL_merra2.txt', sep='\\s+', names=['MJD', 'east_merra2', 'north_merra2', 'up_merra2'])

syog_era5ib.head()

,MJD,east_ib,north_ib,up_ib
0,43874.000000,-1.622,0.437,-4.722
1,43874.041667,-1.605,0.436,-4.605
2,43874.083333,-1.583,0.419,-4.462
3,43874.125000,-1.570,0.397,-4.357
4,43874.166667,-1.547,0.373,-4.231


In [9]:
# Convert MJD to datetime 
for df in [aboa_era5ib , aboa_era5tugo, aboa_ecco2, aboa_merra2,
          syog_era5ib , syog_era5tugo, syog_ecco2, syog_merra2,
          vesl_era5ib , vesl_era5tugo, vesl_ecco2, vesl_merra2] :
    df['date'] = df['MJD'].apply(lambda x: mjd_to_datetime(x) if pd.notna(x) else None)

# Calculate mean for NTL since there are multiple measurements per day
dataframes = [aboa_era5ib, aboa_era5tugo, aboa_ecco2, aboa_merra2,
              syog_era5ib, syog_era5tugo, syog_ecco2, syog_merra2,
              vesl_era5ib, vesl_era5tugo, vesl_ecco2, vesl_merra2]

for i, df in enumerate(dataframes):
    df = df.groupby(df['date'].dt.date, as_index=False).mean()
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    dataframes[i] = df

# Unpack them back to original variables
(aboa_era5ib, aboa_era5tugo, aboa_ecco2, aboa_merra2,
 syog_era5ib, syog_era5tugo, syog_ecco2, syog_merra2,
 vesl_era5ib, vesl_era5tugo, vesl_ecco2, vesl_merra2) = dataframes


aboa_era5ib.head()

,MJD,east_ib,north_ib,up_ib,date
0,43874.479167,-1.033375,0.314417,-4.557375,1979-01-01
1,43875.479167,-0.668333,0.278625,-2.239875,1979-01-02
2,43876.479167,-0.671958,0.261292,-2.374833,1979-01-03
3,43877.479167,-0.591375,0.115167,-2.043875,1979-01-04
4,43878.479167,-0.670292,0.124875,-3.004958,1979-01-05


In [10]:
# ESMGFZ
# Function to read csv, convert to mm, and add MJD
def read_and_convert(file_path, column_names):
    df = pd.read_csv(file_path, sep=',', names=column_names, skiprows=1)
    
    # Ensure last three columns are numeric before multiplying
    df.iloc[:, -3:] = df.iloc[:, -3:].apply(pd.to_numeric, errors='coerce') * 1000  

    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%dT%H:%M:%SZ')
    df['MJD'] = df['date'].apply(datetime_to_mjd)
    
    return df

# Column names for each dataset
column_names_ntal = ['date', 'lat', 'lon', 'east_ntal', 'north_ntal', 'up_ntal']
column_names_ntol = ['date', 'lat', 'lon', 'east_ntol', 'north_ntol', 'up_ntol']
column_names_hydl = ['date', 'lat', 'lon', 'east_hydl', 'north_hydl', 'up_hydl']

# Read and process data
aboa_ntal = read_and_convert('ESMGFZ/ABOA/ABOA_NTAL_combined.csv', column_names_ntal)
aboa_ntol = read_and_convert('ESMGFZ/ABOA/ABOA_NTOL_combined.csv', column_names_ntol)
aboa_hydl = read_and_convert('ESMGFZ/ABOA/ABOA_HYDL_combined.csv', column_names_hydl)

syog_ntal = read_and_convert('ESMGFZ/SYOG/SYOG_NTAL_combined.csv', column_names_ntal)
syog_ntol = read_and_convert('ESMGFZ/SYOG/SYOG_NTOL_combined.csv', column_names_ntol)
syog_hydl = read_and_convert('ESMGFZ/SYOG/SYOG_HYDL_combined.csv', column_names_hydl)

vesl_ntal = read_and_convert('ESMGFZ/VESL/VESL_NTAL_combined.csv', column_names_ntal)
vesl_ntol = read_and_convert('ESMGFZ/VESL/VESL_NTOL_combined.csv', column_names_ntol)
vesl_hydl = read_and_convert('ESMGFZ/VESL/VESL_HYDL_combined.csv', column_names_hydl)

aboa_ntal.head()

,date,lat,lon,east_ntal,north_ntal,up_ntal,MJD
0,1995-01-01 00:00:00,-73.043771,-13.407135,0.195832,-0.409075,-0.140590,49718.000
1,1995-01-01 03:00:00,-73.043771,-13.407135,0.246023,-0.488023,-0.521849,49718.125
2,1995-01-01 06:00:00,-73.043771,-13.407135,0.264395,-0.515332,-0.639675,49718.250
3,1995-01-01 09:00:00,-73.043771,-13.407135,0.273604,-0.549885,-0.782826,49718.375
4,1995-01-01 12:00:00,-73.043771,-13.407135,0.272998,-0.582403,-1.000604,49718.500


In [11]:
dataframes = [aboa_ntal, aboa_ntol, aboa_hydl,
              syog_ntal, syog_ntol, syog_hydl, 
              vesl_ntal, vesl_ntol, vesl_hydl]


for i, df in enumerate(dataframes):
    df = df.groupby(df['date'].dt.date, as_index=False).mean()
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    dataframes[i] = df

# Unpack them back to original variables
(aboa_ntal, aboa_ntol, aboa_hydl,
 syog_ntal, syog_ntol, syog_hydl, 
 vesl_ntal, vesl_ntol, vesl_hydl) = dataframes

aboa_ntal.head()

,date,lat,lon,east_ntal,north_ntal,up_ntal,MJD
0,1995-01-01,-73.043771,-13.407135,0.245344,-0.540710,-0.795427,49718.4375
1,1995-01-02,-73.043771,-13.407135,0.198378,-0.522821,-1.343760,49719.4375
2,1995-01-03,-73.043771,-13.407135,0.269387,-0.450088,-1.837074,49720.4375
3,1995-01-04,-73.043771,-13.407135,0.279554,-0.319965,-0.906681,49721.4375
4,1995-01-05,-73.043771,-13.407135,0.508997,-0.288251,-0.868127,49722.4375


# NTL corrections
This section applies NTL corrections to the GNSS time series. The procedure involves the following steps:

1. **Data merging**: Base GNSS datasets (GR, TUD, OSU, NGL, AY) are merged with relevant auxiliary datasets to enable automated processing.
2. **NTL corrections**: Eleven different NTL correction combinations are applied. Some corrections are specific to the GR dataset, which already includes certain NTL adjustments.
3. **Saving results**: Corrected time series are stored in a combined dictionary (`merged_all`) and exported as CSV files for each station and dataset.


In [12]:
# Metadata for processing
base_keys = ['gr', 'tud', 'osu', 'ngl', 'ay']
merge_suffix = {
    'gr': ['ecco2', 'ntol', 'hydl'],
    'tud': ['ib', 'tugo', 'ecco2', 'merra2', 'ntal', 'ntol', 'hydl'],
    'osu': ['ib', 'tugo', 'ecco2', 'merra2', 'ntal', 'ntol', 'hydl'],
    'ngl': ['ib', 'tugo', 'ecco2',  'merra2', 'ntal', 'ntol', 'hydl'], 
    'ay': ['ib', 'tugo', 'ecco2',  'merra2', 'ntal', 'ntol', 'hydl']
}
colnames = {
    'gr': ['east', 'north', 'up'],
    'tud': ['east', 'north', 'up'],
    'osu': ['east', 'north', 'up'],
    'ngl': ['east', 'north', 'up'],
    'ay': ['east', 'north', 'up']
}

# Example input structure: aboa_gr, aboa_tud, aboa_ntal, etc.
stations = ['aboa', 'syog', 'vesl']
all_datasets = {
    'aboa': {
        'gr': aboa_gr, 'tud': aboa_tud, 'osu': aboa_osu, 'ay': aboa_ay,
        'ib': aboa_era5ib, 'tugo': aboa_era5tugo, 'ecco2': aboa_ecco2,
        'merra2': aboa_merra2, 'ntal': aboa_ntal, 'ntol': aboa_ntol,
        'hydl': aboa_hydl  
    },
    'syog': {
        'gr': syog_gr, 'tud': syog_tud, 'osu': syog_osu, 'ngl': syog_ngl, 'ay': syog_ay,
        'ib': syog_era5ib, 'tugo': syog_era5tugo, 'ecco2': syog_ecco2,
        'merra2': syog_merra2, 'ntal': syog_ntal, 'ntol': syog_ntol,
        'hydl': syog_hydl
    },
    'vesl': {
        'gr': vesl_gr, 'tud': vesl_tud, 'osu': vesl_osu, 'ngl': vesl_ngl, 'ay':vesl_ay,
        'ib': vesl_era5ib, 'tugo': vesl_era5tugo, 'ecco2': vesl_ecco2,
        'merra2': vesl_merra2, 'ntal': vesl_ntal, 'ntol': vesl_ntol,
        'hydl': vesl_hydl
    }
}

In [13]:
# Final results
merged_all = {}

for base_key in base_keys:
    merged_results = {}

    for station in stations:
        dfs = all_datasets.get(station, {})

        if base_key not in dfs:
            print(f"{station}_{base_key} not available — skipping.")
            continue

        try:
            df_base = dfs[base_key].copy()
            df_base['date'] = pd.to_datetime(df_base['date'])
            base_cols = colnames[base_key]
            cols_to_use = ['date', 'MJD'] + base_cols if 'MJD' in df_base.columns else ['date'] + base_cols
            merged = df_base[cols_to_use].copy()
        except Exception as e:
            print(f"Error processing {station}_{base_key}: {e}")
            continue

        for suffix in merge_suffix[base_key]:
            if suffix not in dfs:
                print(f"{station}_{suffix} not available — skipping.")
                continue

            try:
                df = dfs[suffix].copy()
                df['date'] = pd.to_datetime(df['date'])
                merge_cols = [f'east_{suffix}', f'north_{suffix}', f'up_{suffix}']
                available_cols = [col for col in merge_cols if col in df.columns]
                merged = pd.merge(
                    merged,
                    df[['date'] + available_cols],
                    on='date',
                    how='left'
                )
            except Exception as e:
                print(f"Error merging {station}_{suffix} into {station}_{base_key}: {e}")
                continue

        merged_results[station] = merged

    merged_all[base_key] = merged_results


aboa_ngl not available — skipping.


In [14]:
# Dataset-specific correction suffixes
gr_only_corrections = {'_c_ecco2', '_c_ntol', '_c_all_b'}
other_datasets_corrections = {
    '_c_ib', '_c_tugo', '_c_merra2', '_c_ntal',
    '_c_ibecco2', '_c_merra2ecco2', '_c_ntalntol', '_c_all'
}

# Define all correction rules
corrections = [
    ('_c_ecco2', ['east', 'north', 'up'], ['east_ecco2', 'north_ecco2', 'up_ecco2']),
    ('_c_ntol', ['east', 'north', 'up'], ['east_ntol', 'north_ntol', 'up_ntol']),
    ('_c_all_b', ['east', 'north', 'up'], ['east_ntol', 'north_ntol', 'up_ntol', 'east_hydl', 'north_hydl', 'up_hydl']),

    ('_c_ib', ['east', 'north', 'up'], ['east_ib', 'north_ib', 'up_ib']),
    ('_c_tugo', ['east', 'north', 'up'], ['east_tugo', 'north_tugo', 'up_tugo']),
    ('_c_merra2', ['east', 'north', 'up'], ['east_merra2', 'north_merra2', 'up_merra2']),
    ('_c_ntal', ['east', 'north', 'up'], ['east_ntal', 'north_ntal', 'up_ntal']),
    ('_c_ibecco2', ['east', 'north', 'up'], ['east_ib', 'east_ecco2', 'north_ib', 'north_ecco2', 'up_ib', 'up_ecco2']), 
    ('_c_merra2ecco2', ['east', 'north', 'up'], ['east_merra2', 'east_ecco2', 'north_merra2', 'north_ecco2', 'up_merra2', 'up_ecco2']),
    ('_c_ntalntol', ['east', 'north', 'up'], ['east_ntal', 'east_ntol', 'north_ntal', 'north_ntol', 'up_ntal', 'up_ntol']),
    ('_c_all', ['east', 'north', 'up'], 
     ['east_ntal', 'east_ntol', 'east_hydl','north_ntal', 'north_ntol', 'north_hydl','up_ntal', 'up_ntol', 'up_hydl']),
]

In [15]:
for dataset_key, station_dict in merged_all.items():  # e.g., 'gr', 'ngl'
    if dataset_key == 'gr':
        valid_suffixes = gr_only_corrections
    else:
        valid_suffixes = other_datasets_corrections

    for station, df in station_dict.items():  # e.g., 'aboa', 'syog'
        if df is None or not isinstance(df, pd.DataFrame):
            print(f"{dataset_key}_{station} is not a valid DataFrame — skipping")
            continue

        for suffix, coords, corr_cols in corrections:
            if suffix not in valid_suffixes:
                continue

            for i, coord in enumerate(coords):
                col_name = f"{coord}{suffix}"
                base_col = coord

                # One-term correction
                if len(corr_cols) == 3:
                    term_col = corr_cols[i]
                    if term_col in df.columns:
                        df[col_name] = df[base_col] - df[term_col]
                    else:
                        print(f"{col_name} skipped: missing {term_col} in {dataset_key}_{station}")
                else:
                    # Multi-term correction
                    terms = [t for t in corr_cols if t.startswith(coord)]
                    if all(t in df.columns for t in terms):
                        df[col_name] = df[base_col] - sum(df[t] for t in terms)
                    else:
                        print(f"{col_name} skipped: missing terms for {coord} in {dataset_key}_{station}")


In [16]:
for dataset_key, station_dict in merged_all.items():
    for station, df in station_dict.items():
        if df is not None and isinstance(df, pd.DataFrame):
            df.to_csv(f"GNSS/corrected/{station}_{dataset_key}_corrections.csv", index=False)


# Save data
For Hector analysis, the GNSS time series are converted into `.mom` files. Each file corresponds to a single station, dataset, correction, and displacement component. Filenames follow the convention `XXYY_Z.mom`, where `XX` encodes the station and dataset, `YY` identifies the correction (with `00` for uncorrected), and `Z` indicates the component (`0` = east, `1` = north, `2` = up). Each file begins with a comment line (`# sampling period 1`) followed by the MJD and displacement data without headers.


In [17]:
# Function to write a .mom file with a comment at the first row
def write_mom_with_comment(df, filename):
    comment = 'sampling period 1'

    # Determine fractional part from first timestamp in data
    first_mjd = df.iloc[0, 0]  # First value in MJD column
    frac_part = first_mjd - int(first_mjd)

    # Default steps
    steps = []
    if filename[21] == 'A':
        steps = [
            "2003-02-01", "2007-12-31", "2008-01-03", "2009-02-04", "2009-02-05",
            "2010-01-12", "2010-02-05", "2011-01-11", "2011-01-13", "2012-01-15",
            "2012-01-17", "2013-01-09", "2013-01-10", "2014-01-23", "2014-01-24",
            "2015-01-05", "2015-01-06", "2016-01-05", "2016-01-06", "2017-02-02",
            "2017-02-06"
        ]

    elif filename[21] == 'S':
        steps = ["2007-01-26", "2013-12-23", "2020-01-18"]

    elif filename[21] == 'V':
        steps = ["2012-01-08", "2012-08-21"]

    with open(filename, 'w') as f:
        f.write(f"# {comment}\n")

        # Write Hector offsets with same fractional part as first observation
        for step in steps:
            mjd = datetime_to_mjd(step) + frac_part
            f.write(f"# offset {mjd:.6f}\n")

    # Append the data without headers
    df.to_csv(filename, sep=' ', index=False, header=False, float_format='%.6f', mode='a')


In [18]:
# Map station and dataset to 1-letter or 2-letter codes
station_map = {
    'aboa': 'A',
    'syog': 'S',
    'vesl': 'V'
}
dataset_map = {
    'gr': 'G',
    'ngl': 'N',
    'tud': 'T',
    'osu': 'O',
    'ay': 'A'
}
correction_map = {
    'ecco2': '01',
    'ntol': '02',
    'all_b': '03',
    'ib': '04',
    'tugo': '05',
    'merra2': '06',
    'ntal': '07',
    'ibecco2': '08',
    'merra2ecco2': '09',
    'ntalntol': '10',
    'all': '11'
}

# Components and output
components = ['east', 'north', 'up']
output_dir = 'hector/mom_files_all'

# Loop through nested merged_all structure
for dataset, station_dict in merged_all.items():
    for station, df in station_dict.items():
        if df is None or not isinstance(df, pd.DataFrame):
            continue

        st_code = station_map.get(station, station[:2].upper())
        ds_code = dataset_map.get(dataset, dataset[:2].upper())
        base_code = st_code + ds_code 

        # Save original components
        for i, comp in enumerate(components):
            if comp in df.columns:
                filename = os.path.join(output_dir, f"{base_code}00_{i}.mom")
                write_mom_with_comment(df[['MJD', comp]], filename)

        # Save corrected components
        for col in df.columns:
            if any(col.startswith(f"{comp}_c_") for comp in components):
                comp_type, _, corr = col.partition('_c_')
                idx = components.index(comp_type)
                corr_code = correction_map.get(corr, corr[:2].upper())
                filename = os.path.join(output_dir, f"{base_code}{corr_code}_{idx}.mom")
                write_mom_with_comment(df[['MJD', col]], filename)